In [28]:
import random
import string
from pathlib import Path
import os
import requests
import shutil

from git import Repo
from bs4 import BeautifulSoup as Soup
import openai
from PIL import Image

In [29]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [30]:
PATH_TO_BLOG_REPO = Path("/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator/.git")

In [31]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [32]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [33]:
PATH_TO_CONTENT

PosixPath('/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator/content')

In [34]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [35]:
def random_string(length):
    letters = string.ascii_lowercase + "\n"
    
    return ''.join(random.choice(letters) for i in range(length))

random_string(10)

'sclxvnqbss'

In [11]:
def update_blog(commit_message='Updates blog'):
    # GitPython -- Repo locations
    repo = Repo(PATH_TO_BLOG_REPO)
    # git add .
    repo.git.add(all=True)
    # git commit -m "updates blog"
    repo.index.commit(commit_message)
    # git push
    origin = repo.remote(name='origin')
    origin.push()

In [12]:
random_text_string = "kdsaflkjiewoasvantluat"

In [13]:
with open(PATH_TO_BLOG/"index.html", 'w') as f:
    f.write(random_text_string)

In [14]:
update_blog()

In [15]:
def create_new_blog(title,content,cover_image):
    cover_image= Path(cover_image)
    
    # auto file naming utility
    files = len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image,PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        # WRITE A NEW HTML FILE
        with open(path_to_new_content,"w") as f:
            f.write("<!DOCTYPE html>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>\n")
            
            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>")
            # OpenAI --->> Completion GPT -->> "hello\nblog post\n"
            f.write(content.replace("\n", "<br />\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("Blog created")
            return path_to_new_content
    else:
        raise FileExistsError("File already exists, please check again your name! Aborting!")

In [16]:
path_to_new_content = create_new_blog('Test_title','thisisatest','templogo.png')

Blog created


In [17]:
# index.html --> Blog posts

In [19]:
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())

In [20]:
str(soup)

'kdsaflkjiewoasvantluat'

In [21]:
# checking duplicate links

In [22]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    # /pathtocontent/1.html check for overwriting
    return content_path in urls

In [25]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/"index.html") as index:
        soup = Soup(index.read())

    links = soup.find_all("a")
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link does already exist!")
        
    link_to_new_blog = soup.new_tag("a", href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog.string = path_to_new_content.name.split(".")[0]
    last_link.insert_after(link_to_new_blog)
    
    with open(PATH_TO_BLOG/"index.html", "w") as f:
        f.write(str(soup.prettify(formatter='html')))

In [26]:
write_to_index(path_to_new_content)

IndexError: list index out of range

In [ ]:
update_blog()

In [ ]:
def create_prompt(title):
    prompt = """
    Biography: My name is Kristin and I am a Software Engineer with a emphasis on Devops practices.
    
    Blog
    Title: {}
    tags: tech, devops, AI, python, observability, coding
    Summary: I talk about what the future of AI could hold for Devops
    Full Text: """.format(title)
    return prompt

In [ ]:
title = "The future of devops and AI"
print(create_prompt(title))

In [ ]:
response = openai.Completion.create(engine='text-davinci-003',
                                    prompt=create_prompt(title),
                                    max_tokens=1000,
                                    temperature=0.7)

In [ ]:
blog_content = response['choices'][0]['text']

In [ ]:
print(blog_content)

In [ ]:
def dalle2_prompt(title):
    prompt = f"pixel digital art with no text showing {title}"
    return prompt

In [ ]:
image_prompt =  dalle2_prompt(title)

In [ ]:
image_prompt

In [ ]:
response = openai.Image.create(prompt=image_prompt,
                              n=1,size="1024x1024")

In [ ]:
image_url = response['data'][0]['url']

In [ ]:
image_url

In [ ]:
from PIL import Image

In [ ]:
from pillow import Image

In [ ]:
from PIL import Image

In [ ]:
def save_image(image_url, file_name):
    image_res = requests.get(image_url, stream = True)
    
    if image_res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(image_res.raw, f)
    else:
        print("Error downloading image!")
    return image_res.status_code

In [ ]:
import requests
import shutil

In [ ]:
save_image(image_url,file_name="title2.png")